# 自動運転を学習する

Stable BaselinesとRL Baselines Zooを用いて、強化学習による自動運転を試します。

こちらのブログ([Learning to Drive Smoothly in Minutes – Towards Data Science](https://towardsdatascience.com/learning-to-drive-smoothly-in-minutes-450a7cdb35f4))の内容をステップ・バイ・ステップで再現してみます。

ただし、本ノートブックは**Colaboratoryだけで実行することができません。みなさんのMac, Windows, Linuxローカル環境=お手元のPC
を使って試す必要がありますのでご注意ください。**

このノートブックは以下の内容を含みます。

- 環境準備
- Gym環境とエージェントを作成
- エージェントの学習と評価
- 1行のコマンドで学習
- リプレイ


## A. 環境を準備する

Stable Baselinesと依存ライブラリをインストールします。

### 1. ローカル環境にPythonをセットアップ

今回使うのはPython 3.x系です。セットアップには色々な方法があります。ここでは、JupyterLab, Jupyter Notebooksや、他によく使う科学技術計算系パッケージをまとめてくれている[Anaconda](https://www.anaconda.com/)を使ったインストールを紹介します。

もともと、ご自身でPython 3.x系環境をセットアップされている場合、こちらのステップは飛ばしてください。また、pyenvなど他の方法でセットアップしても問題ありません。


#### WIndowsの場合

[Anaconda を Windows にインストールする手順 | WEB ARCH LABO](https://weblabo.oscasierra.net/python-anaconda-install-windows/)などの記事を参考に、セットアップします。


#### Mac OSXの場合

[Anaconda を macOS にインストールする手順 | WEB ARCH LABO](https://weblabo.oscasierra.net/python-anaconda-install-macos/)などの記事を参考に、セットアップします。


### 2. ノートブックをダウンロードし、ローカル環境で開く

`ファイル > .ipynbをダウンロード` から、このノートブックをダウンロードします。

適当なパスに保存したら、Anaconda NavigatorからJupyterLab、またはJupyter Notebookを起動し、ノートブックを開きましょう。コマンドラインから起動しても構いません。

### 3. 必要なライブラリのインストール

このセルを、ローカル環境で立ち上げたJupyterで開いてますか？それではローカル環境に、以下および依存するライブラリを導入します。

In [0]:
!pip install tensorflow stable_baselines pygame

### 2. learning-to-drive-in-5minutesのコピー

gitがインストールされていたら、以下のコマンドで [araffin/learning-to-drive-in-5-minutes](https://github.com/araffin/learning-to-drive-in-5-minutes) のGitHubリポジトリ内容を、ローカルにコピーします。

gitがインストールされていない場合は、上のリポジトリトップページから、zipでダウンロードし、ノートブックと同じ階層に、 `learning-to-drive-in-5-minutes` というディレクトリを作り、保存します(zipは、同ディレクトリに、必要ファイルが入った状態で解凍されるはずです)。

In [0]:
!git clone https://github.com/araffin/learning-to-drive-in-5-minutes.git
%cd learning-to-drive-in-5-minutes

## B. Gym環境を準備する

Donkey Simulatorという、自動運転のシミュレーション環境をセットアップします。

以下のいずれかの方法で、みなさんのお手元のマシンで動かせる状態にします。

- ビルド済みの実行ファイルをダウンロードする
  - [tawnkramer/sdsandbox at donkey](https://github.com/tawnkramer/sdsandbox/tree/donkey)のソースから、筆者がMac用/ Windows用の実行ファイルをビルドしたものです
	  - [Donkey Simulator(Mac OSX版)のダウンロード](https://github.com/tomo-makes/dl-in-a-sec/blob/master/binaries/donkeysim_mac.zip)
	  - [Donkey Simulator(Windows版)のダウンロード](https://github.com/tomo-makes/dl-in-a-sec/blob/master/binaries/donkeysim_win.zip)
    
   - [araffin/learning-to-drive-in-5-minutes](https://github.com/araffin/learning-to-drive-in-5-minutes)のGitHubリポジトリには、[Linux用のバイナリ](https://drive.google.com/open?id=1h2VfpGHlZetL5RAPZ79bhDRkvlfuB4Wb)も公開されています(筆者は動作未確認です)
- ソースからビルドする
  - Unityをインストールする
  - Unityで[tawnkramer/sdsandbox at donkey](https://github.com/tawnkramer/sdsandbox/tree/donkey)のプロジェクトファイル、ソースコードを読み込む
  - 対象のプラットフォーム向けにビルドする

## C. その他の処理を準備する

学習済みモデルをダウンロードします。
再び、[araffin/learning-to-drive-in-5-minutes: Implementation of reinforcement learning approach to make a car learn to drive smoothly in minutes](https://github.com/araffin/learning-to-drive-in-5-minutes)から、以下をダウンロードします。

- Level 0環境用
  - [学習済みVAE](https://drive.google.com/open?id=1n7FosFA0hALhuESf1j1yg-hERCnfVc4b)
  - [学習済みエージェント](https://drive.google.com/open?id=10Hgd5BKfn1AmmVdLlNcDll6yXqVkujoq)

**学習済みVAE**は、 `vae-level-0-dim-32.pkl` という名前に変更しておきましょう。

## D. モデルを作成する

stable-baselinesを使ったモデルの作成は、さきほどコピーしたリポジトリ内の、train.py内に記述されています。

```
model = ALGOS[args.algo].load(args.trained_agent, env=env, tensorboard_log=tensorboard_log, verbose=1, **hyperparams)
```

コマンド実行時の引数で、モデルを選ぶことができます。

## E. エージェントを学習させる

まずSimulatorを起動し、その後にエージェントの学習コマンドを実行します。Simulatorは適切な画面に、予め遷移しておく必要があります。

### 1. Donkey Simulatorを起動し、準備する

先ほど準備した実行ファイルを起動します。

1. 最初に解像度の選択画面が現れます。任意の解像度を選びます。小さすぎなければ、 `640 x 480` を選び、画質を `Fastest` とすると、最も軽く動作します。また、Windowed(フルスクリーンではなくウィンドウでの実行)にチェックを付けておくと、ノートブックを実行しながら状況を確認しやすいです
2. `Play!` ボタンで開始します
3. メニュー画面が表示されます
4. 一度 `Generated Road` を押し、道路と車が表示されることを確認します

ここまで進んだら、次のエージェントの学習コマンドを実行します。

> **注意:** その後、もう一度画面操作が必要です。メニュー画面に遷移した直後の状態を読み取り、学習が開始するようなので、先にSimulatorを起動すると4.のステップのあと `EXIT` でメニューに戻る必要があります。または、以下のエージェント学習のコマンドを実行し、Simuatorへの接続待ち状態でDonkey Simulatorを起動すると、起動後すぐに学習が始まります。



### 2. エージェントを学習させる

試し学習を進めてみます。まず次のセルを実行しましょう。

すると、以下のメッセージが繰り返し、セルの出力に表示される状態になります。

```
Waiting for sim to start...if the simulation is running, press EXIT to go back to the menu
```

そうなったら、先ほど起動したSimulator画面で、右下の `EXIT` を押し、メインメニューに戻りましょう。すると、

```
Got a new client ('127.0.0.1', xxxx)
Scene Selection Ready
```

と表示され、自動的に学習が始まります。

学習中、コースアウトをしたあと走り続けることが続く場合（何らかのバグのようです）、右下の `EXIT` ボタンを押すと、メインメニューから自動的にドライブモードに戻り、学習が継続します。

学習を進める間にも、みるみる運転が上達することがわかります。10分以内に学習が完了します。


In [0]:
!python train.py --algo sac -vae vae-level-0-dim-32.pkl 

## F. 学習済みエージェントの評価

### 1. 学習済みエージェントの振る舞いを見る

学習済みエージェントの運転を観察してみましょう。うまく学習が行われたでしょうか。

In [0]:
!python enjoy.py --algo sac -vae vae-level-0-dim-32.pkl --exp-id 0 -n 2000

## まとめと発展

このノートブックでは、仮想環境における自動運転を試してみました。

Level 0環境だけでなく、Level 1環境で試したい場合は、同ディレクトリの `config.py` で、 `LEVEL = 0` を、 `LEVEL = 1` に書き換えることで試せます。

また、Donkey Carを制作すると、ここで学習したモデルを実際にラジコンに積み込んで動かすこともできます。